In [ ]:
pip install -q python-dotenv

In [9]:
pip install -q openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
# loading the OpenAI API key
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [14]:
# importing the necessary libraries
import openai
import re
import httpx
import os

In [15]:
from openai import OpenAI
client = OpenAI()

MODEL = 'gpt-4o-mini' 
prompt = 'Write something short but funny.'

chat_completion = client.chat.completions.create(
    model=MODEL,
    messages=[{'role': 'user', 'content': prompt}]
)

In [16]:
chat_completion.choices[0].message.content

'Why did the scarecrow win an award? Because he was outstanding in his field!'

## Creating the Agent Class

In [2]:
class Agent:
    def __init__(self, system=''): 
        self.system = system 
        self.messages = []
        
        if self.system:
            self.messages.append({'role': 'system', 'content': system})

    def __call__(self, prompt):
        self.messages.append({'role': 'user', 'content': prompt})
        result = self.execute()
        self.messages.append({'role': 'assistant', 'content': result})
        return result

    def execute(self, model='gpt-4o-mini', temperature=0):
        completion = client.chat.completions.create(
                        model=model, 
                        temperature=temperature,
                        messages=self.messages)
        
        return completion.choices[0].message.content

In [3]:
prompt = '''
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_cost:
e.g. get_cost: book
returns the cost of a book

wikipedia:
e.g. wikipedia: LangChain
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session #1:

Question: How much does a pen cost?
Thought: I should look the pen cost using get_cost
Action: get_cost: pen
PAUSE

You will be called again with this:

Observation: A pen costs $5

You then output:

Answer: A pen costs $5


Example session #2

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
'''.strip()

## Creating the Tools

In [8]:
import httpx
# 1. the calculate() function takes in a string, evaluates that string, and returns the result
def calculate(what):
    return eval(what)

# 2. the get_cost() function returns the cost for a pen, a book, and a stapler
def get_cost(thing):
    if thing in 'pen': 
        return('A pen costs $5')
    elif thing in 'book':
        return('A book costs $20')
    elif thing in 'stapler':
        return('A stapler costs $10')
    else:
        return('A random thing for writing costs $12.')

# 3. the wikipedia() function uses the Wikipedia API to search for a specific query on Wikipedia
def wikipedia(q):
    response = httpx.get('https://en.wikipedia.org/w/api.php', params={
        'action': 'query',
        'list': 'search',
        'srsearch': q,
        'format': 'json'
    })
    results = response.json().get('query').get('search', [])
    
    if not results:
        return None
    return results[0]['snippet']

In [9]:
pip install httpx -q

Note: you may need to restart the kernel to use updated packages.


In [10]:
wikipedia('langchain')

'<span class="searchmatch">LangChain</span> is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework'